In [1]:
# Install library
!pip install pandas
!pip install matplotlib
!pip install sklearn
!pip install h5py
!pip install tslearn
!pip install selenium
!pip install webdriver_manager

In [2]:
# Modules for crawling
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

In [3]:
# Chrome service
service = ChromeService(executable_path=ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\glenn\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [4]:
# Collect symbols
url = 'https://quotes.vcbs.com.vn/a/exchange.html?symbol=HSX'
filename = 'symbols.csv'

driver = webdriver.Chrome(service=service)
driver.get(url)
sleep(3)
body = driver.find_element(By.TAG_NAME, 'body')
div = body.find_elements(By.CLASS_NAME, 'StockSymbolEven')

symbols = []
with open(filename, 'w') as f:
    f.write('Symbols\n')
    for item in div:
        text = item.text
        f.write(text + '\n')
        symbols.append(text)
f.close()
print(len(symbols))

driver.quit()

204


In [ ]:
# Collect closing price 2020, 2021, 2022
from IPython.display import clear_output
import random as rd
import os

url = 'https://s.cafef.vn/Lich-su-giao-dich-VNINDEX-1.chn'
driver = webdriver.Chrome(service=service)
driver.get(url)
sleep(1)
used_symbols = []
needed_symbol_quantity = 100
years = [2020, 2021, 2022]
msg = ''
msg_tmp = ''
times = 0
step = 0
os.system('mkdir data')
while len(used_symbols) < needed_symbol_quantity:
    try:
        if step % 2 == 0:
            msg += msg_tmp
            msg_tmp = ''
        symbol = symbols[rd.randint(0, len(symbols) - 1)]
        while symbol in used_symbols:
            symbol = symbols[rd.randint(0, len(symbols) - 1)]
        h1 = driver.find_element(By.TAG_NAME, 'h1')
        if len(used_symbols) > 0 and used_symbols[-1] in h1.text and times != 0:
            symbolTxt = driver.find_elements(By.TAG_NAME, 'input')[3]
            driver.execute_script("document.getElementsByTagName('input')[3].value = ''")
            symbolTxt.send_keys(symbol)
            sleep(1)
            label = driver.find_element(By.CLASS_NAME, 'ac_over')
            label.click()
        else:
            try:
                symbolTxt = driver.find_elements(By.TAG_NAME, 'input')[4]
                driver.execute_script("document.getElementsByTagName('input')[4].value = ''")
                symbolTxt.send_keys(symbol)
                sleep(1)
                label = driver.find_element(By.CLASS_NAME, 'ac_over')
                label.click()
            except:
                symbolTxt = driver.find_elements(By.TAG_NAME, 'input')[1]
                driver.execute_script("document.getElementsByTagName('input')[1].value = ''")
                symbolTxt.send_keys(symbol)
                sleep(1)
                label = driver.find_element(By.CLASS_NAME, 'ac_over')
                label.click()
            times += 1
        table = driver.find_elements(By.TAG_NAME, 'table')
        if len(table) > 1:
            sleep(1)
            flag = True
            index = 1
            filename = './data/{}.csv'.format(symbol)
            with open(filename, 'w') as f:
                f.write('Date,Closing Price\n')
                while flag:
                    sleep(1.5)
                    clear_output()
                    print('{}{}Crawling... {} at page {}'.format(msg, msg_tmp, symbol, index))
                    tbody = driver.find_elements(By.TAG_NAME, 'tbody')
                    if len(tbody) < 3:
                        used_symbols = used_symbols[:-1]
                        break
                    trs = tbody[1].find_elements(By.TAG_NAME, 'tr')
                    for tr in trs[2:]:
                        td = tr.find_elements(By.TAG_NAME, 'td')
                        year = int(td[0].text.strip()[-4:])
                        if year in years:
                            try:
                                f.write('{},{}\n'.format(td[0].text.strip(), td[2].text.strip()))
                            except:
                                flag = False
                                break
                        else:
                            flag = False
                            break
                    td = tbody[2].find_elements(By.TAG_NAME, 'td')
                    a = td[len(td) - 1].find_elements(By.TAG_NAME, 'a')
                    try:
                        a[len(a) - 1].click()
                    except:
                        break
                    index += 1
            f.close()
            sleep(0.5)
            if index >= 25:
                used_symbols.append(symbol)
                times = 0
                step += 1
                msg_tmp = 'Symbol: {} - Pages: {} - Symbol total: {}\n'.format(symbol, index, len(used_symbols))
            else:
                os.system('rm ./data/{}.csv'.format(used_symbols[-1]))
                used_symbols = used_symbols[:-1]
    except Exception as exception:
        if  exception.__class__.__name__ == 'StaleElementReferenceException':
            os.system('rm ./data/{}.csv'.format(used_symbols[-1]))
            used_symbols = used_symbols[:-1]
        continue
print(len(used_symbols))
driver.quit()

Crawling... HPG at page 30


In [ ]:
# Assign data to dictionary (key is date)
from datetime import date

data = {}
for symbol in used_symbols:
    filename = './data/' + symbol + '.csv'
    not_none = []
    
    with open(filename, 'r') as f:
        for line in f.readlines()[1:]:
            values = line.strip().split(',')
            if values[0] not in data.keys():
                data[values[0]] = [float(values[1])]
            else:
                data[values[0]].append(float(values[1]))
            not_none.append(values[0])
    f.close()
    for key in data.keys():
        if key not in not_none:
            data[key].append(0.0)
today = date.today().strftime('%d/%m/%Y')
data.pop(today)
print(len(data))

In [ ]:
# Remove unnecessary item
for key in data.keys():
    if len(data[key]) > 100:
        data[key] = data[key][:-(len(data[key]) - 100)]

In [ ]:
# Dictionary to dataframe
import pandas as pd

df = pd.DataFrame.from_dict(data, orient='index', columns=used_symbols)
df.head(10)

In [ ]:
df[::-1].to_csv('ClosingPrice.csv')

In [ ]:
# Read data
import pandas as pd

filename = 'ClosingPrice.csv'
df = pd.read_csv(filename)
df.head(10)

In [ ]:
# Normalize data
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

normalizeData = []
for colName in df.columns:
  tssmv = TimeSeriesScalerMeanVariance()
  normalizeData.append((colName, tssmv.fit_transform([df['DCL']])[0]))

In [ ]:
# Show image 5 symbol before normalize and after normalize
import matplotlib.pyplot as plt
import numpy as np

for i in range(5):
  x = np.array([i for i in range(len(normalizeData[i][1]))])
  y = np.array(df[normalizeData[i][0]])
  y2 = np.array(normalizeData[i][1])
  fig, ax = plt.subplots()
  ax.plot(x, y, label='before normalizing')
  ax.plot(x, y2, label='after normalizing')
  ax.set_title(normalizeData[i][0])
  ax.legend()

In [ ]:
# Train test split
from sklearn.model_selection import train_test_split

X_train
y_train
X_test

In [ ]:
# Standard Euclidean k-means
from tslearn.clustering import TimeSeriesKMeans

